# Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

In [1]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# List Pipeline Execution Steps


In [2]:
%store -r pipeline_name

In [3]:
print(pipeline_name)

BERT-pipeline-1621102217


In [4]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:298039562326:pipeline/bert-pipeline-1621102217/execution/34f5wo3bayx8',
  'PipelineExecutionDisplayName': 'execution-1621102228350',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2021, 5, 15, 18, 10, 28, 189000, tzinfo=tzlocal())}]
CPU times: user 10.7 ms, sys: 1.26 ms, total: 12 ms
Wall time: 106 ms


In [5]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [6]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:298039562326:pipeline/bert-pipeline-1621102217/execution/34f5wo3bayx8


In [7]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'EndTime': datetime.datetime(2021, 5, 15, 18, 48, 17, 743000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:298039562326:model-package/bert-reviews-1621102219/1'}},
                             'StartTime': datetime.datetime(2021, 5, 15, 18, 48, 16, 822000, tzinfo=tzlocal()),
                             'StepName': 'RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'EndTime': datetime.datetime(2021, 5, 15, 18, 48, 17, 529000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:298039562326:model/pipelines-34f5wo3bayx8-createmodel-bbjbc6kn0t'}},
                             'StartTime': datetime.datetime(2021, 5, 15, 18, 48, 16, 820000, tzinfo=tzlocal()),
                             'StepName': 'CreateModel',
                             'StepStatus': 'Succeeded'},
                  

# View Registered Model

In [8]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

arn:aws:sagemaker:us-east-1:298039562326:model-package/bert-reviews-1621102219/1


In [9]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

# View Created Model

In [10]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

arn:aws:sagemaker:us-east-1:298039562326:model/pipelines-34f5wo3bayx8-createmodel-bbjbc6kn0t
pipelines-34f5wo3bayx8-createmodel-bbjbc6kn0t


# Create Model Endpoint from Model Registry
More details here:  https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html


In [11]:
import time

timestamp = int(time.time())

model_from_registry_name = "bert-model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

Model from registry name : bert-model-from-registry-1621110419


In [12]:
from pprint import pprint

create_model_from_registry_response = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:298039562326:model/bert-model-from-registry-1621110419',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '97',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Sat, 15 May 2021 20:26:59 GMT',
                                      'x-amzn-requestid': '16804f76-ad74-4d8d-a314-da7b92eed2a1'},
                      'HTTPStatusCode': 200,
                      'RequestId': '16804f76-ad74-4d8d-a314-da7b92eed2a1',
                      'RetryAttempts': 0}}


In [13]:
model_from_registry_arn = create_model_from_registry_response["ModelArn"]
model_from_registry_arn

'arn:aws:sagemaker:us-east-1:298039562326:model/bert-model-from-registry-1621110419'

In [14]:
endpoint_config_name = "bert-model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.4xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": pipeline_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

bert-model-from-registry-epc-1621110419


In [15]:
%store -r pipeline_endpoint_name

try:
    print("Using existing Pipeline EndpointName: {}".format(pipeline_endpoint_name))
    
except NameError:
    timestamp = int(time.time())
    pipeline_endpoint_name = "bert-model-from-registry-ep-{}".format(timestamp)
    print("Created Pipeline EndpointName={}".format(pipeline_endpoint_name))

    create_endpoint_response = sm.create_endpoint(
        EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
    )
    print(create_endpoint_response["EndpointArn"])

no stored variable or alias pipeline_endpoint_name
Created Pipeline EndpointName=bert-model-from-registry-ep-1621110420
arn:aws:sagemaker:us-east-1:298039562326:endpoint/bert-model-from-registry-ep-1621110420


In [16]:
%store pipeline_endpoint_name

Stored 'pipeline_endpoint_name' (str)


In [17]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_

In [18]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

CPU times: user 162 ms, sys: 17 ms, total: 179 ms
Wall time: 6min 31s


# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# Predict the star_rating with Ad Hoc review_body Samples

In [19]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

In [20]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is awful."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

Predicted star_rating: {'predicted_label': 5}
Predicted star_rating: {'predicted_label': 1}


# Release Resources

In [21]:
# sm.delete_endpoint(
#      EndpointName=pipeline_endpoint_name
# )

In [22]:
# %%html

# <p><b>Shutting down your kernel for this notebook to release resources.</b></p>
# <button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

# <script>
# try {
#     els = document.getElementsByClassName("sm-command-button");
#     els[0].click();
# }
# catch(err) {
#     // NoOp
# }
# </script>